# Import thư viện

In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf
import regex as re
from nltk.corpus import stopwords
from keras.models import model_from_json
from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Flatten, Dense, Softmax, Dropout

In [3]:
movie = pd.read_csv('..\Data\movie_info.csv')

In [4]:
movie.shape

(33838, 23)

In [6]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33838 entries, 0 to 33837
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Title                     33838 non-null  object 
 1   Tomatometer score         33838 non-null  int64  
 2   Audience score            30302 non-null  float64
 3   Tomatometer count         33838 non-null  object 
 4   Audience count            33838 non-null  object 
 5   Tomatometer state         33838 non-null  object 
 6   Audience state            30302 non-null  object 
 7   Genre                     33016 non-null  object 
 8   Original Language         32682 non-null  object 
 9   Director                  33678 non-null  object 
 10  Producer                  28287 non-null  object 
 11  Writer                    26653 non-null  object 
 12  Release Date (Theaters)   19694 non-null  object 
 13  Box Office (Gross USA)    13216 non-null  object 
 14  Runtim

In [10]:
movie.isnull().sum().sort_values()

Title                           0
Tomatometer score               0
Tomatometer count               0
Audience count                  0
Tomatometer state               0
Cast                          149
Director                      160
Genre                         822
Original Language            1156
Runtime                      1230
Synopsis                     1269
Audience score               3536
Audience state               3536
Release Date (Streaming)     5203
Producer                     5551
Writer                       7185
Release Date (Theaters)     14144
Distributor                 16439
Rating                      18171
Box Office (Gross USA)      20622
Sound Mix                   24092
Aspect Ratio                28438
View the collection         33333
dtype: int64

# Tiền xử lí dữ liệu

## Tiền xử lí "Rating" và "Genre" bằng mô hình dự báo

In [17]:
def preprocess_text(text):
    text = text.lower() # Chuyển văn bản thành chữ thường
    text = re.sub(r'[^\w\s]',' ',text) # Xóa dấu câu
    text = re.sub("\d+", " ", text) # Xóa chữ số
    stop = stopwords.words('english') # Xóa các từ không có nghĩa (Stop Words)
    text = " ".join(text for text in text.split() if text not in stop)
    return text

In [19]:
# Tiền xử lí cột "Synopsis" để đưa vào mô hình
for i in range(len(movie)):
    movie['Synopsis'].iloc[i] = preprocess_text(movie['Synopsis'].iloc[i])

C:\Users\Nguyen Quang Tu\AppData\Local\Temp\ipykernel_12904\889473171.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie['Synopsis'].iloc[i] = preprocess_text(movie['Synopsis'].iloc[i])


AttributeError: 'float' object has no attribute 'lower'

### Tiền xử lí cột "Rating"

In [14]:
# Load model predict Rating đã train
json_file = open('Predict Rating Model\predict_rating.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("Predict Rating Model\predict_rating.h5")

In [16]:
t = """
In 1974, after years of civil war, the Portuguese and their descendants fled the colony of Angola where independentist groups gradually claimed their territory back. TOMMY GUNS gives a dual perspective on this struggle: a tribal girl who discovers love and death when her path crosses that of a young Portuguese soldier; and a group of Portuguese soldiers who are barracked inside an infinite wall from which they will have to escape once the past comes out of the grave to claim its long awaited justice.
"""

x_train, x_test, y_train, y_test = train_test_split(movie['Synopsis'].values, movie['Rating'].values, test_size = 0.2, random_state = 0)
vectorizer = TfidfVectorizer().fit(x_train)

t = vectorizer.transform([preprocess_text(t)])
np.argmax(loaded_model.predict(t.todense()),axis=1)

NameError: name 'genre' is not defined